# LAB | GenAI: Exploring Prompting Techniques for Customer Support Automation

## Tasks

**Objective:**

Learn and apply different prompting techniques to improve the performance of a language model in generating customer support responses.

**Business Case:**

Imagine you are working for a company that provides a variety of services, including technical support, billing inquiries, and general customer service. Your task is to use a language model to automate responses to customer emails.

**Dataset:**

Download the FAQ of a company to do this exercise. Below you have a couple of examples, but feel free to find your own:
 - https://info.undp.org/erecruit/documents/FAQ.pdf
 - https://www.cambridgeenglish.org/Images/696254-faqs-digital-cambridge-english-qualifications.pdf
 - https://www.wscc.nt.ca/sites/default/files/documents/0009-518-Item-04-INDESIGN-FAQ-Template%203%20-%20MINUS%20FIRST%20QUESTION.pdf


### Task 1

Download and Read the PDF:

  - Choose one of the provided FAQ PDFs or find your own relevant FAQ document.
  - Read through the FAQ document carefully to understand the types of questions and answers it contains.
  - Create Questions Based on the PDF ( you can use ChatGPT for this)
    - Generate a list of potential customer questions that could be answered using the information from the FAQ PDF.
    - Ensure your questions cover a variety of topics and difficulty levels found within the document.
    - Generate Responses Using Different Prompting Techniques:

Use a language model (such as ChatGPT) to generate responses to your questions.
Experiment with different prompting techniques to see how they affect the quality of the responses.

## Types of prompting

For each of the types prompting, perform the following:
 - Research what the type of prompting is
 - Create a small explaination of the prompting
 - Test your type of prompting vs the control prompt (direct question)

### Zero-Shot Prompting

Use the knowledge base to create prompts without examples.
Test the model's ability to generate accurate responses based solely on the provided instructions.
Assess the performance compared to few-shot prompting.

In [4]:
from openai import OpenAI
import sys
import subprocess
from dotenv import load_dotenv
from openai import OpenAI
import os


load_dotenv()

True

In [5]:
model_gpt = "gpt-4o-mini"

client = OpenAI(
    # This is the default and can be omitted
    api_key=os.getenv("OPENAI_API_KEY"),
)

In [7]:
# Define a function for zero-shot prompting
def zero_shot_prompt(question):
    response = client.chat.completions.create(
        model="gpt-4",
        messages=[
            {"role": "system", "content": "You are an assistant trained to answer customer support questions based on the given context."},
            {"role": "user", "content": question}
        ]
    )
    return response.choices[0].message.content

# List of questions based on the FAQ
questions = [
    "What is the UNDP eRecruit system?",
    "Can someone apply for a job on the UNDP eRecruit system without meeting the minimum requirements?",
    "What steps should I take if I forgot my password for the UNDP eRecruit system?",
    "How can I check the status of my job application?",
    "Is it possible to revise my application after submission?",
]

# Generate responses for each question
responses = {q: zero_shot_prompt(q) for q in questions}

# Print the questions and corresponding responses
for question, response in responses.items():
    print(f"Question: {question}\nResponse: {response}\n{'-'*50}")


Question: What is the UNDP eRecruit system?
Response: The UNDP eRecruit system is an online platform used by the United Nations Development Programme for recruitment purposes. It allows prospective candidates to create a profile, apply for jobs, and track the status of their applications. It serves as a central system for all job vacancies within the UNDP, streamlining the job application process for both the applicants and the hiring team.
--------------------------------------------------
Question: Can someone apply for a job on the UNDP eRecruit system without meeting the minimum requirements?
Response: No, you cannot apply for a job on the UNDP eRecruit system without meeting the minimum requirements. The system is designed to only accept applications from individuals who meet the minimum specifications mentioned in the job posting. Submitting an application without meeting these requirements may result in the application being automatically disqualified, so it's recommended to tho

### Few-Shot Prompting

Select a few representative emails from each category.
Create prompts by including these examples and ask the model to generate responses for new emails.
Evaluate the quality and relevance of the responses.

In [8]:
# Define a function for few-shot prompting
def few_shot_prompt(new_question):
    examples = [
        {
            "question": "What is the UNDP eRecruit system?",
            "answer": "The UNDP eRecruit system is an online platform that allows applicants to create and manage personal profiles for submitting job applications to various UNDP job postings."
        },
        {
            "question": "Can someone apply for a job on the UNDP eRecruit system without meeting the minimum requirements?",
            "answer": "No, applicants cannot apply for a job posting unless they meet the specified minimum requirements, such as education, work experience, and language skills."
        },
        {
            "question": "What steps should I take if I forgot my password for the UNDP eRecruit system?",
            "answer": "If you forgot your password, you should click the 'Forgot User ID or Password' link on the login page. Follow the instructions provided to reset your password."
        }
    ]

    # Format the examples for the prompt
    few_shot_context = "\n".join(
        [f"Q: {ex['question']}\nA: {ex['answer']}" for ex in examples]
    )

    # Add the new question to the few-shot context
    prompt = f"""
You are a customer support assistant. Based on the examples provided, generate a response for the new question.

Examples:
{few_shot_context}

New Question:
Q: {new_question}
A:
    """
        # Generate the response
    response = client.chat.completions.create(
        model="gpt-4",
        messages=[
            {"role": "system", "content": "You are an assistant trained to answer customer support questions based on the given examples."},
            {"role": "user", "content": prompt}
        ]
    )
    return response.choices[0].message.content

# New questions to test the few-shot prompting
new_questions = [
    "How can I check the status of my job application?",
    "Is it possible to revise my application after submission?",
    "Does UNDP charge any fees during the recruitment process?",
]

# Generate responses for each new question
new_responses = {q: few_shot_prompt(q) for q in new_questions}

# Print the new questions and corresponding responses
for question, response in new_responses.items():
    print(f"Question: {question}\nResponse: {response}\n{'-'*50}")



Question: How can I check the status of my job application?
Response: You can check the status of your job application by logging into your account on the UNDP eRecruit system. Once logged in, navigate to the 'Application History' section where you can view the status of all your applications.
--------------------------------------------------
Question: Is it possible to revise my application after submission?
Response: No, once a job application is submitted via the UNDP eRecruit system, you cannot revise it. It's advisable to thoroughly review your application before submission to ensure all details are accurate and up-to-date.
--------------------------------------------------
Question: Does UNDP charge any fees during the recruitment process?
Response: No, the UNDP does not charge any fees at any stage of the recruitment process. All recruitment processes are free of charge. Be aware of scams that ask for payments.
--------------------------------------------------


### Chain of Thought Prompting

Develop prompts that guide the model to think through the problem step-by-step before providing the final answer.
Analyze if this approach improves the quality of technical support responses.


In [9]:
# Define a function for Chain of Thought (CoT) prompting
def chain_of_thought_prompt(question):
    prompt = f"""
You are a customer support assistant. When answering the question, think step-by-step through the problem and explain the reasoning before providing the final answer.

Question: {question}

Step-by-step reasoning:
1.
    """
    # Generate the response
    response = client.chat.completions.create(
        model="gpt-4",
        messages=[
            {"role": "system", "content": "You are an assistant trained to answer customer support questions step-by-step."},
            {"role": "user", "content": prompt}
        ]
    )
    return response.choices[0].message.content

# New questions to test Chain of Thought prompting
new_questions = [
    "How can I check the status of my job application?",
    "What should I do if I forgot my password for the UNDP eRecruit system?",
    "Can I apply for multiple job postings simultaneously?",
]

# Generate responses for each new question
responses = {q: chain_of_thought_prompt(q) for q in new_questions}

# Print the questions and corresponding responses
for question, response in responses.items():
    print(f"Question: {question}\nResponse: {response}\n{'-'*50}")


Question: How can I check the status of my job application?
Response: To check the status of your job application, you first need to understand where you have submitted your job application. If you have applied through a job portal, an employer's website or through a recruiting agency, the steps to follow might be a little different.

2. If you applied through a job portal like Indeed, LinkedIn, or Glassdoor, here's what you may need to do:

   a. Log onto the job portal where you submitted your application.
   
   b. Navigate to your account or profile section, where you should be able to see your submitted applications.
   
   c. Click on the specific application that you are interested in. There should be an option to view its status, but this may vary in different portals.

3. If you applied directly through an employer's website, you may have received an acknowledgement email embedded with an application status link. You can check that email for the status link or follow these ste

### Instruction-Based Prompting

Write clear and explicit instructions in the prompts for each type of customer inquiry.
Measure the effectiveness of detailed instructions in guiding the model.

In [12]:
# Define a function for Instruction-Based Prompting
def instruction_based_prompt(question, detailed_instructions):
    prompt = f"""
You are a customer support assistant. Follow the detailed instructions provided to answer the question accurately and clearly.

Instructions:
{detailed_instructions}

Question: {question}

Answer:
    """
    # Generate the response
    response = client.chat.completions.create(
        model="gpt-4",
        messages=[
            {"role": "system", "content": "You are an assistant trained to follow detailed instructions to answer customer inquiries."},
            {"role": "user", "content": prompt}
        ]
    )
    return response.choices[0].message.content

# Define detailed instructions
detailed_instructions = """
1. Identify the key elements of the question (e.g., specific features, processes, or requirements mentioned).
2. Cross-check these elements with the FAQ or known information.
3. If the question involves a procedure, provide a step-by-step guide.
4. Ensure the tone is professional, clear, and concise.
5. If relevant, include links or resources from the FAQ for further clarification.
"""

# New questions to test Instruction-Based Prompting
new_questions = [
    "How can I check the status of my job application?",
    "What browsers are compatible with the UNDP eRecruit system?",
    "Can I apply for multiple job postings simultaneously?",
]

# Generate responses for each new question
responses = {q: instruction_based_prompt(q, detailed_instructions) for q in new_questions}

# Print the questions and corresponding responses
for question, response in responses.items():
    print(f"Question: {question}\nResponse: {response}\n{'-'*50}")


Question: How can I check the status of my job application?
Response: Thank you for your query regarding the status of your job application. Based on the instructions provided, here is the information you need:

1. Visit our online application system on our website 
2. Login using your account credentials: the username and password created when you submitted your application.
3. After logging in, you'll be able to view your application dashboard. Your application status is typically indicated under the section "status" or "application status". 

Please keep in mind the time frames for application processing may vary depending on the role you applied for and the number of applicants we received. 

If you're having trouble logging in or if you have any other questions, refer to our FAQ page here [insert FAQ link] or feel free to get in touch with our support team. Thank you for your patience and your interest in joining our team.
--------------------------------------------------
Questio

### Role-Playing Prompting

Ask the model to respond as a customer service representative or technical support expert.
Evaluate how well the model adopts the role and provides relevant information.

In [13]:
# Define a function for Role-Playing Prompting
def role_playing_prompt(question):
    prompt = f"""
You are a highly skilled customer service representative for UNDP. Your job is to provide accurate and helpful responses to customer inquiries in a professional and friendly tone.

Customer Inquiry: {question}

Response:
    """
    # Generate the response
    response = client.chat.completions.create(
        model="gpt-4",
        messages=[
            {"role": "system", "content": "You are an expert customer service representative for UNDP, helping users resolve their inquiries with professionalism and clarity."},
            {"role": "user", "content": prompt}
        ]
    )
    return response.choices[0].message.content

# New questions to test Role-Playing Prompting
new_questions = [
    "How do I access the UNDP eRecruit system?",
    "What should I do if I encounter issues logging into my account?",
    "Is there a fee for submitting applications through the UNDP eRecruit system?",
]

# Generate responses for each question
responses = {q: role_playing_prompt(q) for q in new_questions}

# Print the questions and corresponding responses
for question, response in responses.items():
    print(f"Question: {question}\nResponse: {response}\n{'-'*50}")

Question: How do I access the UNDP eRecruit system?
Response: Dear Customer,

Thank you for reaching out and showing interest in the UNDP eRecruit system.

To access the UNDP eRecruit system, please follow the below steps:

1. Visit UNDP's official website.
2. Navigate to the 'Careers' section usually found in the header or footer navigation menus.
3. Under the Careers section, you'll see a category named 'Job Opportunities' or similar. When you click on it, it will redirect you to the job search portal.
4. In the job search portal, you should find a link to the eRecruit system. If you are a new user, you'll need to register an account with your email ID first.
5. After registering and confirming your email address, you can log in to access the eRecruit system.

Please note that the eRecruit system is where you can find job vacancies, submit your applications, review application status, and manage your profile.

If you encounter any problems or need further assistance, please do not he

### Contextual Prompting

Provide relevant context from previous email threads or the knowledge base before posing the main question.
Test if providing context improves the accuracy and relevance of the responses.

In [14]:
# Define a function for Contextual Prompting
def contextual_prompt(context, question):
    prompt = f"""
You are a customer support assistant. Use the provided context to answer the question accurately and clearly.

Context:
{context}

Customer Inquiry: {question}

Response:
    """
    # Generate the response
    response = client.chat.completions.create(
        model="gpt-4",
        messages=[
            {"role": "system", "content": "You are a customer support assistant trained to use context effectively for accurate responses."},
            {"role": "user", "content": prompt}
        ]
    )
    return response.choices[0].message.content

# Example context from a knowledge base or email thread
context = """
The UNDP eRecruit system is an online platform for managing job applications. 
All applications must be submitted online using this system. Offline or email submissions are not accepted.
Applicants must meet the minimum requirements for each job posting, including education, work experience, and language skills.
Password recovery is available through the "Forgot User ID or Password" link.
"""

# New questions to test Contextual Prompting
new_questions = [
    "What should I do if I forgot my password?",
    "Can I apply for a job through email if I meet the minimum requirements?",
    "How do I check if I meet the minimum requirements for a job posting?",
]

# Generate responses for each question
responses = {q: contextual_prompt(context, q) for q in new_questions}

# Print the questions and corresponding responses
for question, response in responses.items():
    print(f"Question: {question}\nResponse: {response}\n{'-'*50}")

Question: What should I do if I forgot my password?
Response: You can recover your password by clicking on the "Forgot User ID or Password" link in the UNDP eRecruit system. Follow the instructions provided to retrieve your password.
--------------------------------------------------
Question: Can I apply for a job through email if I meet the minimum requirements?
Response: No, you cannot apply for a job through email. All applications must be submitted online using the UNDP eRecruit system. Offline or email submissions are not accepted. Please ensure you meet the minimum requirements before applying.
--------------------------------------------------
Question: How do I check if I meet the minimum requirements for a job posting?
Response: You can check if you meet the minimum requirements for a job posting by reviewing the job description and requirements thoroughly on the UNDP eRecruit system. Each job posting will have specific details about the education, work experience, and langua

### Conversational Prompting

Create a dialogue-style prompt where the model continues an ongoing conversation with the customer.
Observe how well the model maintains context and coherence in multi-turn conversations.

In [15]:
def conversational_prompt(conversation_history, new_message):
    messages = [
        {"role": "system", "content": "You are a helpful and professional customer service representative for UNDP. Maintain the context and provide coherent responses in the conversation."}
    ]
    # Add the conversation history to the prompt
    for history in conversation_history:
        messages.append({"role": history["role"], "content": history["content"]})
    
    # Add the new customer message
    messages.append({"role": "user", "content": new_message})
    
    # Generate the response
    response = client.chat.completions.create(
        model="gpt-4",
        messages=messages
    )
    return response.choices[0].message.content

# Example conversation history
conversation_history = [
    {"role": "user", "content": "How do I access the UNDP eRecruit system?"},
    {"role": "assistant", "content": "You can access the UNDP eRecruit system via this link: https://undpcareers.partneragencies.org/erecruit.html."},
    {"role": "user", "content": "What if I encounter issues logging in?"}
]

# New customer message in the ongoing conversation
new_message = "Can I apply for a job without meeting the minimum requirements?"

# Generate a response for the new message
response = conversational_prompt(conversation_history, new_message)

# Print the conversation with the new response
for turn in conversation_history:
    print(f"{turn['role'].capitalize()}: {turn['content']}")
print(f"Assistant: {response}")

User: How do I access the UNDP eRecruit system?
Assistant: You can access the UNDP eRecruit system via this link: https://undpcareers.partneragencies.org/erecruit.html.
User: What if I encounter issues logging in?
Assistant: At UNDP, we take qualifications seriously for ensuring we have the right fit for the role. The minimum requirements outlined in the job description are crucial, and we typically encourage applicants to only apply for jobs where they meet these. However, if you believe you possess the necessary skills or relevant experience that may substitute the minimum requirements, you might still consider applying, but you should elaborate on this in your application. Please note that meeting the minimum requirements does not guarantee an interview or job offer. All applications go through a stringent review process. If your profile matches the requirements for a specific role, our recruitment team will contact you.


### Contrastive Prompting

Show the model examples of both good and bad responses.
Use these contrasting examples to guide the model towards generating better responses.
Compare the results with other techniques.

In [16]:
# Define a function for Contrastive Prompting
def contrastive_prompt(question):
    prompt = f"""
You are a customer service representative. Use the examples of good and bad responses below to guide your response.

Examples:
Bad Response:
Q: How do I access the UNDP eRecruit system?
A: Just go online and search for it. You’ll find it.

Good Response:
Q: How do I access the UNDP eRecruit system?
A: You can access the UNDP eRecruit system via this link: https://undpcareers.partneragencies.org/erecruit.html. Make sure to use a compatible browser such as Google Chrome or Internet Explorer 9 in compatibility view mode.

Bad Response:
Q: What if I forgot my password for the eRecruit system?
A: Reset it or something.

Good Response:
Q: What if I forgot my password for the eRecruit system?
A: If you forgot your password, click the 'Forgot User ID or Password' link on the login page and follow the instructions to reset your password.

Now, using these examples, provide a helpful and professional response to the following question:

Q: {question}

Response:
    """
    # Generate the response
    response = client.chat.completions.create(
        model="gpt-4",
        messages=[
            {"role": "system", "content": "You are an assistant trained to generate helpful and professional responses by learning from examples of good and bad responses."},
            {"role": "user", "content": prompt}
        ]
    )
    return response.choices[0].message.content

# New questions to test Contrastive Prompting
new_questions = [
    "Can I apply for a job without meeting the minimum requirements?",
    "What should I do if I encounter issues logging into my account?",
    "Is there a fee for submitting applications through the UNDP eRecruit system?",
]

# Generate responses for each question
responses = {q: contrastive_prompt(q) for q in new_questions}

# Print the questions and corresponding responses
for question, response in responses.items():
    print(f"Question: {question}\nResponse: {response}\n{'-'*50}")

Question: Can I apply for a job without meeting the minimum requirements?
Response: A: While you are certainly welcome to attempt to apply for any position, it is strongly recommended that you only apply for jobs where you meet the minimum requirements. This is to ensure the best fit for both the applicant and the organization. However, do take into account any transferrable skills or equivalent experiences you may have that could potentially fulfill those requirements.
--------------------------------------------------
Question: What should I do if I encounter issues logging into my account?
Response: A: I'm sorry to hear that you're experiencing issues logging into your account. First, ensure your username and password are correct. If you've forgotten your password, you can click on the 'Forgot User ID or Password' link on the login page and follow the instructions to reset it. If the problem persists, please contact our support team for further assistance.
--------------------------

### Specificity Prompting

Ask the model to respond with a specific style, tone, or level of detail, such as formal, friendly, or concise.
Assess how well the model adapts its responses to the specified requirements.

In [17]:
# Define a function for Specificity Prompting
def specificity_prompt(question, tone="formal", level_of_detail="detailed"):
    prompt = f"""
You are a customer service representative for UNDP. Respond to the customer inquiry in a {tone} tone and provide a {level_of_detail} explanation.

Customer Inquiry: {question}

Response:
    """
    # Generate the response
    response = client.chat.completions.create(
        model="gpt-4",
        messages=[
            {"role": "system", "content": "You are a customer service representative trained to adjust your tone and detail level as required."},
            {"role": "user", "content": prompt}
        ]
    )
    return response.choices[0].message.content

# New questions to test Specificity Prompting
new_questions = [
    "How do I access the UNDP eRecruit system?",
    "What should I do if I forgot my password for the UNDP eRecruit system?",
    "Is there a fee for submitting applications through the UNDP eRecruit system?",
]

# Define tones and detail levels to test
tones = ["formal", "friendly", "concise"]
detail_levels = ["detailed", "brief"]

# Generate responses for each tone and detail level
responses = {}
for tone in tones:
    for detail_level in detail_levels:
        key = f"{tone.capitalize()} & {detail_level.capitalize()}"
        responses[key] = {
            q: specificity_prompt(q, tone=tone, level_of_detail=detail_level)
            for q in new_questions
        }

# Print the questions and corresponding responses
for style, answers in responses.items():
    print(f"\nStyle: {style}\n{'-'*50}")
    for question, response in answers.items():
        print(f"Question: {question}\nResponse: {response}\n{'-'*50}")


Style: Formal & Detailed
--------------------------------------------------
Question: How do I access the UNDP eRecruit system?
Response: Dear Inquirer,

Thank you for your interest in using the United Nations Development Programme (UNDP) eRecruit system. Here's a detailed set of instructions to guide you in accessing this service:

1. Launch your preferred internet browser on your device (Chrome, Safari, Firefox, etc.)
2. Enter the UNDP website address in the browser’s URL bar: https://www.undp.org/
3. Locate the "Careers" section usually placed in the headers or footers of the webpage or by utilizing the website's search function.
4. Once you find yourself on the Careers page, look for a section labelled "Vacancies". The UNDP posts all of its job opportunities here.
5. "UNDP's eRecruit system" is accessible from this Vacancies page. There you will find an option to 'View All Vacancies' that will lead you to the eRecruit system.
6. Once you are in the eRecruit system, you can browse 

### Iterative Refinement Prompting

Ask the model to refine or improve upon its previous response.
Experiment with multiple iterations to see if responses improve over time.

In [18]:
# Define a function for Iterative Refinement Prompting
def iterative_refinement_prompt(question, iterations=3):
    initial_prompt = f"""
You are a customer service representative for UNDP. Answer the following question clearly and professionally.

Customer Inquiry: {question}

Response:
    """
    
    # Generate the initial response
    response = client.chat.completions.create(
        model="gpt-4",
        messages=[
            {"role": "system", "content": "You are an expert customer service representative."},
            {"role": "user", "content": initial_prompt}
        ]
    )
    refined_response = response.choices[0].message.content
    
    # Refine the response iteratively
    for i in range(iterations):
        refine_prompt = f"""
The following is a response provided by you to a customer inquiry. Please refine and improve it to make it clearer, more professional, and more helpful.

Previous Response:
{refined_response}

Refined Response:
        """
        response = client.chat.completions.create(
            model="gpt-4",
            messages=[
                {"role": "system", "content": "You are an expert customer service representative trained to refine your responses for clarity and professionalism."},
                {"role": "user", "content": refine_prompt}
            ]
        )
        refined_response = response.choices[0].message.content
    
    return refined_response

# New questions to test Iterative Refinement Prompting
new_questions = [
    "What should I do if I forgot my password for the UNDP eRecruit system?",
    "Can I apply for multiple job postings at the same time?",
    "Is there a fee for submitting applications through the UNDP eRecruit system?",
]

# Generate refined responses for each question
refined_responses = {q: iterative_refinement_prompt(q, iterations=3) for q in new_questions}

# Print the questions and their final refined responses
for question, response in refined_responses.items():
    print(f"Question: {question}\nRefined Response: {response}\n{'-'*50}")

Question: What should I do if I forgot my password for the UNDP eRecruit system?
Refined Response: Dear Esteemed Customer,

Thank you for reaching out regarding the password recovery for our UNDP eRecruit system. We understand how crucial it is to restore your access and are eager to guide you through the process. 

Kindly follow the steps outlined for a seamless process:

1. Navigate to the UNDP eRecruit System Login page.
2. Seek out and select the 'Forgot your password?' option.
3. Accurately input your registered email address in the given field.
4. Click the 'Submit' button post entering your email address.
5. Inspect your email inbox for a password reset mail from our end containing a unique link.
6. Open the email and follow the embedded link, which will guide you to a page designated for creating a new password.

If, at any point, you encounter a challenge during the process or if the password reset email does not reach you promptly, please contact us instantly for further assi